## Practical Exercises

In [1]:
import io
import matplotlib
import matplotlib.pyplot as plt
import numpy
import pandas as pd
import seaborn

seaborn.set_context('talk')

## Reading the dataset

In [2]:
url = 'https://www.famaf.unc.edu.ar/~nocampo043/sysarmy_survey_2020_processed.csv'
df = pd.read_csv(url)

In [3]:
df[:3]

,profile_gender,profile_age,work_country,work_province,profile_years_experience,work_years_in_company,work_years_in_current_position,work_people_in_charge_of,profile_studies_level,profile_studies_level_state,...,salary_has_bonus,salary_inflation_adjustment_2020,salary_percentage_inflation_adjustment_2020,salary_month_last_inflation_adjustment,work_has_violence_situations,profile_has_disabilities_hiring_difficulties,company_employee_number,company_main_activity,company_recommended,company_diversity_policies
0,Female,26,Argentina,Ciudad Autónoma de Buenos Aires,3.0,3.0,3.0,0,University,Ongoing,...,No,No,0.0,0,In my current job,NaN,501-1000,Services / Software Consulting / Digital,7,2
1,Male,29,Argentina,Corrientes,5.0,2.0,2.0,4,University,Ongoing,...,Less than a salary,One,10.0,1,Never,No,201-500,Other industries,8,9
2,Female,22,Argentina,Ciudad Autónoma de Buenos Aires,2.0,0.0,0.0,0,Secondary,Complete,...,One salary,No,0.0,0,In a previous job,No,2001-5000,Other industries,6,9


# Exercise - Descriptive analysis

Answer the question: **What programming languages are ​​associated with the best salaries?**

For it:
1. Select the relevant columns to `salary_monthly_NET`, `work_contract_type`, `tools_programming_language`, and `profile_studies_level`.
2. Select the relevant rows to analyze. This includes removing extreme and erroneous values, but you can also focus the analysis on a sub-population. For example, they can be limited to people with a salary greater than 10,000 pesos or people who work only "Full-Time" or with a certain programming language, but you must justify your choice and reformulate the initial question if necessary.
  * Get a list of the most popular programming languages. Then, decide how many and which ones to select to include in the analysis.
3. Select metrics that help answer the question and methods to analyze them. Choose ONE of the following options:
  * Compare salary distributions for each language using visualizations. As the visualization is the final product, it must be clear and show relevant information.
  * Compare measures of descriptive statistics on salary distribution for each language. Be creative. Descriptive statistics allow us to say things like: "Kotlin programmers mostly earn the top 10% of salaries!" (where *majority* is a slippery middle ground that just means more than 50%). To compare multiple languages, we recommend using visualizations as well.
  * Compare odds. For example: "If you know Python or Java, you have a 30% more chance of winning over 100K".

If you consider it necessary, carry out several iterations. That is, if they find that the distributions of the programming languages ​​they initially selected are not very different, they can redo the analysis using only the programming languages ​​that are different.

In [4]:
# complete here if you want to include more columns
relevant_columns = ['tools_programming_languages',
                    'salary_monthly_NET',
                    'work_contract_type',
                    'profile_studies_level']

### Frequency count of programming languages

The column that contains information about the programming languages used is `tools_programming_languages`. Its values are strings with the selected languages separated by commas.

In [5]:
df.tools_programming_languages[:3]

0                        None
1    HTML, Javascript, Python
2                        None
Name: tools_programming_languages, dtype: object

The following code cells separate these programming languages and count how often they appear.

It is not necessary to understand this code in depth, although it is a good exercise.

In [6]:
# Convert the comma-separated string of languages to a list of string.
# Remove 'None of the previous one' option, spaces and training commas.
def split_languages(languages_str):
  if not isinstance(languages_str, str):
    return []
  # Remove 'other' option
  languages_str = languages_str.lower()\
    .replace('none of the previous one', '') \
    .replace('none', '')
  # Split string into list of items
  # Remove spaces and commas for each item
  return [lang.strip().replace(',', '')
          for lang in languages_str.split()]

# Create a new column with the list of languages
df.loc[:, 'cured_programming_languages'] = df.tools_programming_languages\
    .apply(split_languages)
if 'cured_programming_languages' not in relevant_columns:
    relevant_columns.append('cured_programming_languages') 

# Duplicate each row of df for each programming language
# mentioned in the response.
# We only include in df_lang the columns we are going to analyze later, so we
# don't duplicate innecesary information.
df_lang = df.cured_programming_languages\
    .apply(pd.Series).stack()\
    .reset_index(level=-1, drop=True).to_frame()\
    .join(df[relevant_columns])\
    .rename(columns={0: 'programming_language'})
# Horrible programming style! But a lot of data science code can be written with
# as concatenations of functions (pipelines), and there's no elegant way of
# doing that on Python.
df_lang[:5]

/tmp/ipykernel_41944/3367783713.py:26: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  .apply(pd.Series).stack()\


,programming_language,tools_programming_languages,salary_monthly_NET,work_contract_type,profile_studies_level,cured_programming_languages
1,html,"HTML, Javascript, Python",63000.0,Full-Time,University,"[html, javascript, python]"
1,javascript,"HTML, Javascript, Python",63000.0,Full-Time,University,"[html, javascript, python]"
1,python,"HTML, Javascript, Python",63000.0,Full-Time,University,"[html, javascript, python]"
3,bash/shell,"Bash/Shell, C, CSS, HTML, Java, Javascript, PH...",102000.0,Full-Time,Postgraduate,"[bash/shell, c, css, html, java, javascript, p..."
3,c,"Bash/Shell, C, CSS, HTML, Java, Javascript, PH...",102000.0,Full-Time,Postgraduate,"[bash/shell, c, css, html, java, javascript, p..."


In the `programming_language` column, you will find each language separately. Note that if a response contained 3 languages, such as `"HTML, Javascript, Python"`, the row has been replicated 3 times. Therefore, there are three rows with index 1.

## Relevant language filtering

The following code allows you to select only the rows where the value of the `programming_language` column is found in the `interesting_languages` list.

In [8]:
# Filter out languages that we want to exclude
# Complete here with your selected list.
interesting_languages = ["python", "javascript"]
filtered_df_lang = df_lang[df_lang.programming_language.isin(interesting_languages)]
filtered_df_lang

,programming_language,tools_programming_languages,salary_monthly_NET,work_contract_type,profile_studies_level,cured_programming_languages
1,javascript,"HTML, Javascript, Python",63000.0,Full-Time,University,"[html, javascript, python]"
1,python,"HTML, Javascript, Python",63000.0,Full-Time,University,"[html, javascript, python]"
3,javascript,"Bash/Shell, C, CSS, HTML, Java, Javascript, PH...",102000.0,Full-Time,Postgraduate,"[bash/shell, c, css, html, java, javascript, p..."
3,python,"Bash/Shell, C, CSS, HTML, Java, Javascript, PH...",102000.0,Full-Time,Postgraduate,"[bash/shell, c, css, html, java, javascript, p..."
4,javascript,"CSS, HTML, Javascript, SQL, TypeScript",106000.0,Full-Time,University,"[css, html, javascript, sql, typescript]"
...,...,...,...,...,...,...
6087,python,Python,120000.0,Full-Time,University,[python]
6089,python,"Bash/Shell, Python, SQL",130000.0,Full-Time,University,"[bash/shell, python, sql]"
6090,javascript,".NET, C#, COBOL, CSS, HTML, Javascript, PHP, S...",97000.0,Full-Time,Terciary,"[.net, c#, cobol, css, html, javascript, php, ..."
6093,javascript,"C, C++, HTML, Java, Javascript, PHP, Python, SQL",72.0,Full-Time,Terciary,"[c, c++, html, java, javascript, php, python, ..."
